In [ ]:
import pandas as pd 
from datetime import date, datetime
import yfinance as yf

tickers = ['ADBE', 'GE', 'INTU', 'MS', 'VRTX', 'XOM']

dataframes = []

overall_df = pd.DataFrame() 

count1 = 0
for ticker in tickers:
    
#     data_df = yf.download(ticker, start="2019-01-01", end="2024-01-01", auto_adjust=True)
    
    data_df = pd.read_csv(f'{ticker.lower()}/data/test/OVERALL_prediction_by_model.csv')
    data_df['Date'] = pd.to_datetime(data_df['Date'])
    data_df['Close'] = data_df['close']
    data_df = data_df[['Date', 'Close']]
    data_df = data_df.set_index('Date')    

    dataframes.append(data_df)
    
    if len(overall_df) == 0: 
        overall_df = data_df[['Close']]
        overall_df = overall_df.rename(columns = {'Close': f'{ticker}'})
    else: 
        overall_df = pd.concat([overall_df, data_df['Close']], axis = 1)
        overall_df = overall_df.rename(columns = {'Close': f'{ticker}'})


initial_sum = 10000000 
overall_df = overall_df.reset_index(drop = False)
one_share = initial_sum / 6

num_shares = {} 
for item in overall_df.columns: 
    if item == 'Date': continue 
    num_shares[item] = one_share/overall_df[item][0]     
        

adbe_shares = num_shares['ADBE']
ge_shares = num_shares['GE']
intu_shares = num_shares['INTU']
ms_shares = num_shares['MS']
vrtx_shares = num_shares['VRTX']
xom_shares = num_shares['XOM']

adbe_initial = overall_df['ADBE'][0].item()
ge_initial = overall_df['GE'][0].item()
intu_initial = overall_df['INTU'][0].item()
ms_initial = overall_df['MS'][0].item()
vrtx_initial = overall_df['VRTX'][0].item()
xom_initial = overall_df['XOM'][0].item()

average_returns = overall_df.copy()[['Date']]

for column in overall_df.columns: 
    if column == 'Date': continue 
    
    if column == 'ADBE': tools = adbe_shares
    elif column == 'GE': tools = ge_shares
    elif column == 'INTU': tools = intu_shares
    elif column == 'MS': tools = ms_shares
    elif column == 'VRTX': tools = vrtx_shares
    elif column == 'XOM': tools = xom_shares 
        
    average_returns[column] = overall_df.apply(lambda x: x[column]*tools, axis = 1)
    
average_returns['average_returns'] = average_returns.apply(lambda x: round((x['ADBE'] + x['GE'] + x['INTU'] + x['MS'] + x['VRTX'] + x['XOM'] - initial_sum)/initial_sum, 4), axis = 1)
average_returns = average_returns[['Date', 'average_returns']]
average_returns = average_returns.set_index('Date')

market = yf.download('^gspc', start="2019-01-01", end="2024-01-01", auto_adjust=True)
initial = market['Close'][0]
market['market_returns'] = market.apply(lambda x: (x['Close'] - initial) / initial, axis = 1)
market_returns = market[['market_returns']]

baseline = pd.concat([market_returns, average_returns], axis = 1)
baseline 

In [ ]:
portfolio1 = pd.read_csv('ammended_portfolio_outcome/forecasted/a2c_portfolio.csv')
portfolio1['portfolio_returns'] = portfolio1.apply(lambda x: x['daily_return'] - 1, axis = 1)
portfolio1['Date'] = pd.to_datetime(portfolio1['date'])
portfolio1 = portfolio1.set_index('Date')
portfolio = portfolio1[['portfolio_returns']]
portfolio

combined = pd.concat([portfolio, baseline], axis = 1)
combined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

target = combined.copy() 

fig, ax1 = plt.subplots(figsize=(14, 5))
fig.suptitle('Returns Comparison', fontsize=14)

ax1.plot(target.index, target['portfolio_returns'], '-r', label = 'A2C Optimized Portfolio Returns')
ax1.plot(target.index, target['market_returns'], '-b', label = 'Benchmark Market Returns (S&P500)')
ax1.plot(target.index, target['average_returns'], '-g', label = 'Benchmark Average Returns (Not Optimized)')

ax1.set_ylabel("Returns")
ax1.set_xlabel("Date")
plt.grid()
fig.legend(loc = 'upper left')

In [ ]:
weights_df = pd.read_csv('ammended_portfolio_outcome/forecasted/portfolio_weights_a2c.csv')
weights_df['date'] = pd.to_datetime(weights_df['date'])
weights_df = weights_df.set_index('date')
weights_df

In [ ]:
import numpy as np

index = 0
import matplotlib.pyplot as plt

target = weights_df.copy()

fig = plt.figure(figsize=(14,5))
plt.title(f'Allocation Outcome')

for column in target.columns:
    r = np.round(np.random.rand(),1)
    g = np.round(np.random.rand(),1)
    b = np.round(np.random.rand(),1)
    print([r,g,b])
    plt.plot(target.index, target[column], label = column, color = [r, g, b])


plt.legend(loc="best")
plt.grid()

plt.xlabel("Date")
plt.ylabel("Allocation")

## Vanilla

In [ ]:
vportfolio1 = pd.read_csv('ammended_portfolio_outcome/vanilla/a2c_portfolio.csv')
vportfolio1['portfolio_returns'] = vportfolio1.apply(lambda x: x['daily_return'] - 1, axis = 1)
vportfolio1['Date'] = pd.to_datetime(vportfolio1['date'])
vportfolio1 = vportfolio1.set_index('Date')
vportfolio = vportfolio1[['portfolio_returns']]
vportfolio

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

target = combined.copy() 

fig, ax1 = plt.subplots(figsize=(14, 5))
fig.suptitle('Returns Comparison', fontsize=14)

ax1.plot(vportfolio.index, vportfolio['portfolio_returns'], 'black', label = 'A2C (vMACD) Optimized Portfolio Returns')
ax1.plot(target.index, target['portfolio_returns'], '-r', label = 'A2C (fMACD) Optimized Portfolio Returns')
ax1.plot(target.index, target['market_returns'], '-b', label = 'Benchmark Market Returns (S&P500)')
ax1.plot(target.index, target['average_returns'], '-g', label = 'Benchmark Average Returns (Not Optimized)')
ax1.plot(target.index, target['average_returns'], '-g', label = 'Benchmark Average Returns (Not Optimized)')

ax1.set_ylabel("Returns")
ax1.set_xlabel("Date")
fig.legend(loc = 'upper left')
plt.grid()
